## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-01-05-12-13 +0000,bbc,"Five killed in gas explosions in Iran, officia...",https://www.bbc.com/news/articles/cd9ew5p3zvko...
1,2026-02-01-05-11-50 +0000,startribune,Multiple buildings on fire after Hermantown blast,https://www.startribune.com/multiple-buildings...
2,2026-02-01-05-09-40 +0000,missionlocal,"What’s on now at San Francisco museums, Februa...",https://missionlocal.org/2026/01/whats-on-now-...
3,2026-02-01-05-02-21 +0000,nyt,"The Richest 2026 Players: A.I., Crypto, Pro-Is...",https://www.nytimes.com/2026/02/01/us/politics...
4,2026-02-01-05-01-17 +0000,nyt,Why Is Taking the Rest of the Donetsk Region S...,https://www.nytimes.com/2026/02/01/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2461/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,22
108,ice,20
94,minnesota,11
158,epstein,11
88,judge,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
169,2026-01-31-10-30-00 +0000,wsj,Trump’s pick for Fed chair has long attacked t...,https://www.wsj.com/economy/central-banking/ke...,62
83,2026-01-31-20-52-55 +0000,cbc,"Judge orders U.S. to free 5-year-old boy, fath...",https://www.cbc.ca/news/world/father-son-relea...,59
10,2026-02-01-04-26-00 +0000,wsj,President Trump joked during a speech that he ...,https://www.wsj.com/politics/policy/trump-joke...,59
57,2026-01-31-23-46-19 +0000,nypost,Trump claims Minnesota fraud exceeds $19 billi...,https://nypost.com/2026/01/31/us-news/trump-cl...,56
72,2026-01-31-22-14-54 +0000,bbc,Judge orders release of five-year-old detained...,https://www.bbc.com/news/articles/cm2ygq5p2l3o...,56


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
169,62,2026-01-31-10-30-00 +0000,wsj,Trump’s pick for Fed chair has long attacked t...,https://www.wsj.com/economy/central-banking/ke...
83,59,2026-01-31-20-52-55 +0000,cbc,"Judge orders U.S. to free 5-year-old boy, fath...",https://www.cbc.ca/news/world/father-son-relea...
186,40,2026-01-31-06-31-13 +0000,latimes,Epstein files reveal emails between Ghislaine ...,https://www.latimes.com/california/story/2026-...
139,35,2026-01-31-13-30-00 +0000,nypost,Gaza ‘doctor’ who slammed Israel in NY Times o...,https://nypost.com/2026/01/31/world-news/gaza-...
33,32,2026-02-01-02-00-00 +0000,wsj,An Abu Dhabi royal backed a secret $500 millio...,https://www.wsj.com/politics/policy/spy-sheikh...
63,27,2026-01-31-23-08-20 +0000,bbc,"Andrew should testify to US Congress, Starmer ...",https://www.bbc.com/news/articles/cgrd7kpg9pwo...
35,25,2026-02-01-01-37-34 +0000,nypost,"Sean Penn, Robert De Niro lead Hollywood A-lis...",https://nypost.com/2026/01/31/us-news/hollywoo...
20,23,2026-02-01-03-30-45 +0000,nypost,"Unhinged woman storms NYC restaurant, whacks d...",https://nypost.com/2026/01/31/us-news/woman-st...
57,23,2026-01-31-23-46-19 +0000,nypost,Trump claims Minnesota fraud exceeds $19 billi...,https://nypost.com/2026/01/31/us-news/trump-cl...
13,22,2026-02-01-04-00-00 +0000,wsj,Iran’s Gen Z Helped Propel the Protests. They ...,https://www.wsj.com/world/middle-east/irans-ge...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
